# Imports

In [74]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import dask.dataframe as dd
from pyspark.sql import SparkSession
import multiprocessing as mp
import csv
import datatable as dt
import yaml
from box import Box
import gzip
import os
from subprocess import check_call

# Check which solution is the best for reading data with 2GB+ size

In [2]:
df_pandas = pd.read_csv("Parking_2015.csv")
df_pandas.head()

,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,...,Hydrant Violation,Double Parking Violation,Latitude,Longitude,Community Board,Community Council,Census Tract,BIN,BBL,NTA
0,8002531292,EPC5238,NY,PAS,10/01/2014,21,SUBN,CHEVR,T,20390,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8015318440,5298MD,NY,COM,03/06/2015,14,VAN,FRUEH,T,27790,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7611181981,FYW2775,NY,PAS,07/28/2014,46,SUBN,SUBAR,T,8130,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7445908067,GWE1987,NY,PAS,04/13/2015,19,4DSD,LEXUS,T,59990,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7037692864,T671196C,NY,PAS,05/19/2015,19,4DSD,CHRYS,T,36090,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
chunks = pd.read_csv("Parking_2015.csv", chunksize=100000)
df_pandas_chunks = pd.concat(chunks)
df_pandas_chunks.head()

,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,...,Hydrant Violation,Double Parking Violation,Latitude,Longitude,Community Board,Community Council,Census Tract,BIN,BBL,NTA
0,8002531292,EPC5238,NY,PAS,10/01/2014,21,SUBN,CHEVR,T,20390,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8015318440,5298MD,NY,COM,03/06/2015,14,VAN,FRUEH,T,27790,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7611181981,FYW2775,NY,PAS,07/28/2014,46,SUBN,SUBAR,T,8130,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7445908067,GWE1987,NY,PAS,04/13/2015,19,4DSD,LEXUS,T,59990,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7037692864,T671196C,NY,PAS,05/19/2015,19,4DSD,CHRYS,T,36090,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df_dask = dd.read_csv("Parking_2015.csv")
df_dask.head()

,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,...,Hydrant Violation,Double Parking Violation,Latitude,Longitude,Community Board,Community Council,Census Tract,BIN,BBL,NTA
0,8002531292,EPC5238,NY,PAS,10/01/2014,21,SUBN,CHEVR,T,20390,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8015318440,5298MD,NY,COM,03/06/2015,14,VAN,FRUEH,T,27790,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7611181981,FYW2775,NY,PAS,07/28/2014,46,SUBN,SUBAR,T,8130,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7445908067,GWE1987,NY,PAS,04/13/2015,19,4DSD,LEXUS,T,59990,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7037692864,T671196C,NY,PAS,05/19/2015,19,4DSD,CHRYS,T,36090,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
spark = SparkSession \
    .builder \
    .appName("Exploratory Analysis") \
    .getOrCreate()

parking = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load("Parking_2015.csv")
parking.show(5)

+--------------+--------+------------------+----------+----------+--------------+-----------------+------------+--------------+------------+------------+------------+-----------------------+------------------+------------------+---------------+-----------+--------------+------------+--------------+-------------------+----------------+---------------------------------+------------+-----------+-------------------+--------------------+-----------+------------+--------------------+--------------------------+--------------------+------------------+-------------+---------------------+------------+------------+--------------+-------------------+---------------------+---------------------------------+-----------------+------------------------+--------+---------+---------------+------------------+------------+----+----+----+
|Summons Number|Plate ID|Registration State|Plate Type|Issue Date|Violation Code|Vehicle Body Type|Vehicle Make|Issuing Agency|Street Code1|Street Code2|Street Code3|Vehicl

In [6]:
%%time

df_mp = pd.read_csv("Parking_2015.csv", chunksize=1000)
total_length = 0
for chunk in df_mp:
    total_length += len(chunk)
print(total_length)

11809233
Wall time: 4min 2s


In [7]:
df_csv = csv.DictReader(open("Parking_2015.csv"))
i=0
for row in df_csv:
        print(row)
        i += 1
        if i == 5:
            break

{'Summons Number': '8002531292', 'Plate ID': 'EPC5238', 'Registration State': 'NY', 'Plate Type': 'PAS', 'Issue Date': '10/01/2014', 'Violation Code': '21', 'Vehicle Body Type': 'SUBN', 'Vehicle Make': 'CHEVR', 'Issuing Agency': 'T', 'Street Code1': '20390', 'Street Code2': '29890', 'Street Code3': '31490', 'Vehicle Expiration Date': '01/01/20150111 12:00:00 PM', 'Violation Location': '0007', 'Violation Precinct': '7', 'Issuer Precinct': '7', 'Issuer Code': '345454', 'Issuer Command': 'T800', 'Issuer Squad': 'A2', 'Violation Time': '0011A', 'Time First Observed': '', 'Violation County': 'NY', 'Violation In Front Of Or Opposite': 'F', 'House Number': '133', 'Street Name': 'Essex St', 'Intersecting Street': '', 'Date First Observed': '01/05/0001 12:00:00 PM', 'Law Section': '408', 'Sub Division': 'd1', 'Violation Legal Code': '', 'Days Parking In Effect    ': 'Y Y Y', 'From Hours In Effect': '1200A', 'To Hours In Effect': '0300A', 'Vehicle Color': 'BL', 'Unregistered Vehicle?': '', 'Vehi

In [8]:
df_dt = dt.fread("Parking_2015.csv")
df_dt.head()

,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,…,Community Council,Census Tract,BIN,BBL,NTA
,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,,▪,▪,▪,▪,▪
0,8002531292,EPC5238,NY,PAS,10/01/2014,21,SUBN,CHEVR,T,20390,…,NA,NA,NA,NA,NA
1,8015318440,5298MD,NY,COM,03/06/2015,14,VAN,FRUEH,T,27790,…,NA,NA,NA,NA,NA
2,7611181981,FYW2775,NY,PAS,07/28/2014,46,SUBN,SUBAR,T,8130,…,NA,NA,NA,NA,NA
3,7445908067,GWE1987,NY,PAS,04/13/2015,19,4DSD,LEXUS,T,59990,…,NA,NA,NA,NA,NA
4,7037692864,T671196C,NY,PAS,05/19/2015,19,4DSD,CHRYS,T,36090,…,NA,NA,NA,NA,NA
5,7704791394,JJF6834,PA,PAS,11/20/2014,21,4DSD,NISSA,T,74230,…,NA,NA,NA,NA,NA
6,7541599104,FYP8263,NY,PAS,08/01/2014,21,SUBN,HONDA,T,77530,…,NA,NA,NA,NA,NA
7,7443513951,FZL7450,NY,PAS,07/19/2014,71,4DSD,VOLVO,T,49210,…,NA,NA,NA,NA,NA
8,7426321423,86318MA,NY,COM,09/23/2014,69,VAN,CHEVR,T,21690,…,NA,NA,NA,NA,NA


## Conclusion

<p style="font-size:18px;">
    Based on the executed times, "dask" is the best in this scanrio.
</p>

# Use YAML File

In [79]:
with open("config.yaml", "r") as ymlfile:
    cfg = Box(yaml.safe_load(ymlfile))

In [48]:
def read_data_summary(config):
    data = cfg.input.name+'.'+cfg.input.format
    df_dask = dd.read_csv(data, delimiter=cfg.input.delimiter, dtype=cfg.input.dtypes)
    
    print(f'The size of the file is: {os.stat(data).st_size} Bytes')
    print(f'It has: {len(df_dask.columns)} Columns and {len(df_dask)} Rows')
    return df_dask

In [49]:
df = read_data_summary(cfg)

The size of the file is: 2864071408 Bytes
It has: 51 Columns and 11809233 Rows


In [84]:
def validate_data(raw_df):
    trusted_columns = list(map(lambda x: x.lower(),  cfg.input.columns))
    raw_columns = list(map(lambda x: x.lower(),  raw_df.columns))
    
    trusted_columns = [x.strip(' ') for x in trusted_columns]
    raw_columns = [x.strip(' ') for x in raw_columns]
    
    while(True):
        if len(raw_columns)!=len(trusted_columns):
            print(f'Count of columns are invalid! It should be {len(trusted_columns)}, but it is {len(raw_columns)}')
            return
        if raw_columns.sort()!=trusted_columns.sort():
            print('Columns are invalid!')
            return
        if raw_columns.sort()!=trusted_columns.sort():
            print(f'Columns are invalid!')
            print(f'Columns in Uploaded Dataset: {list(set(raw_columns).difference(trusted_columns))} VS. Columns in Config File: {list(set(trusted_columns).difference(raw_columns))}')
            return
    
    
        output_file = cfg.output.name+"."+cfg.output.format
        output_path = cfg.output.path+output_file
        df.to_csv(output_path, header=None, index=None, sep=cfg.output.delimiter, compression='infer')
        print(f'File is uploaded successfully and written to: {output_file}')
        return
    

In [85]:
validate_data(df)

File is uploaded successfully and written to: trusted_Parking_2015.gz
